# 광주 소방서 정보 가져오기
* '관서명', '주소', '전화번호'

In [52]:
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import math

## 1. 광주검색

In [9]:
url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=keyword&myX=37.5605672&myY=126.9433486&searchKeyword='
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_7976\1194244807.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [10]:
# '광주' 검색
driver.find_element(By.ID, 'searchKeyword').send_keys('광주')
driver.find_element(By.ID, 'fsSearchBtn').click()
time.sleep(3)

## 2. 1페이지 정보만 가져오기

In [19]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [26]:
divs = soup.select('.stations-list li')

In [27]:
len(divs)

10

In [53]:
div = divs[0]

In [37]:
title = div.select_one('.title').text
title

'북부소방서_광주-일곡-119 안전센터'

In [38]:
addr = div.select_one('address')
addr

<address>광주광역시 북구 설죽로 529(일곡동)</address>

In [36]:
tel = div.select_one('.tel').text
tel

'062-606-4743'

In [34]:
lines = []

for div in divs:
    title = div.select_one('.title').text
    addr = div.select_one('address').text
    tel = div.select_one('.tel').text
    
    lines.append([title, addr, tel])

In [35]:
df = pd.DataFrame(lines, columns=['관서명', '주소', '전화번호'])
df

,관서명,주소,전화번호
0,광산소방서_광주-평동-119 안전센터,광주광역시 광산구 월전동 1251,전화공사중
1,광산소방서_광주-첨단-119 안전센터,광주광역시 광산구 임방울대로825번길 16(쌍암동),062-606-4883
2,광산소방서_광주-신가-119 안전센터,광주광역시 광산구 수등로 236(신가동),062-606-4863
3,광산소방서_광주-비아-119 안전센터,광주광역시 광산구 비아로12번길 10(비아동),062-606-4843
4,광산소방서_광주-월곡-119 안전센터,광주광역시 광산구 사암로 276(월곡동),062-606-4823
5,광산소방서_광주-송정-119 안전센터,광주광역시 광산구 상무대로 93-1(도산동),062-606-4803
6,광산소방서_광주-하남-119 안전센터,광주광역시 광산구 하남산단1번로 13(하남동),062-606-4903
7,북부소방서_광주-동림-119 안전센터,광주광역시 북구 북문대로 202(동림동),062-606-4783
8,북부소방서_광주-두암-119 안전센터,광주광역시 북구 삼정로 5(두암동),062-606-4763
9,북부소방서_광주-일곡-119 안전센터,광주광역시 북구 설죽로 529(일곡동),062-606-4743


## 3. 전체 페이지 가져오기

In [39]:
# 2페이지 이동
driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()

In [40]:
# 3, 4페이지 이동
driver.find_element(By.CSS_SELECTOR, '.next_page').click()

In [54]:
# 검색결과 건수 가져오기
strong = soup.select('.stations-local-after > p > strong')[1].string

In [56]:
count = int(strong[0:-1])
count = math.ceil(count/10) # ceil : 입력한 정수를 계산하여 올림처리
count

4

In [77]:
driver.get(url)
time.sleep(3)
driver.find_element(By.ID, 'searchKeyword').send_keys('광주')
driver.find_element(By.ID, 'fsSearchBtn').click()

In [78]:
lines = []
for i in range(count):
    # 페이지 이동
    if i == 0:
        time.sleep(3)
    elif i == 1:
        driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()
        time.sleep(3)
    else:
        driver.find_element(By.CSS_SELECTOR, '.next_page').click()
        time.sleep(3)
    
    # 각 페이지의 데이터 추출
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    divs = soup.select('.stations-list li')
    
    for div in divs:
        title = div.select_one('.title').text
        addr = div.select_one('address').text
        tel = div.select_one('.tel').text

        lines.append([title, addr, tel])

In [79]:
df = pd.DataFrame(lines, columns=['관서명', '주소', '전화번호'])
df

,관서명,주소,전화번호
0,광산소방서_광주-평동-119 안전센터,광주광역시 광산구 월전동 1251,전화공사중
1,광산소방서_광주-첨단-119 안전센터,광주광역시 광산구 임방울대로825번길 16(쌍암동),062-606-4883
2,광산소방서_광주-신가-119 안전센터,광주광역시 광산구 수등로 236(신가동),062-606-4863
3,광산소방서_광주-비아-119 안전센터,광주광역시 광산구 비아로12번길 10(비아동),062-606-4843
4,광산소방서_광주-월곡-119 안전센터,광주광역시 광산구 사암로 276(월곡동),062-606-4823
5,광산소방서_광주-송정-119 안전센터,광주광역시 광산구 상무대로 93-1(도산동),062-606-4803
6,광산소방서_광주-하남-119 안전센터,광주광역시 광산구 하남산단1번로 13(하남동),062-606-4903
7,북부소방서_광주-동림-119 안전센터,광주광역시 북구 북문대로 202(동림동),062-606-4783
8,북부소방서_광주-두암-119 안전센터,광주광역시 북구 삼정로 5(두암동),062-606-4763
9,북부소방서_광주-일곡-119 안전센터,광주광역시 북구 설죽로 529(일곡동),062-606-4743


In [80]:
df['관서명'].nunique()

35

In [81]:
df.to_csv('광주소방서.csv', index=False)